<a href="https://colab.research.google.com/github/Ishant508/Ishant508.github.io/blob/master/doctorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd 
from numba import jit, float32

def distcorr(X, Y):
    """ Compute the distance correlation function
    
    >>> a = [1,2,3,4,5]
    >>> b = np.array([1,2,9,4,4])
    >>> distcorr(a, b)
    0.762676242417
    """
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()
    
    dcov2_xy = (A * B).sum()/float(n * n)
    dcov2_xx = (A * A).sum()/float(n * n)
    dcov2_yy = (B * B).sum()/float(n * n)
    dcor = np.sqrt(dcov2_xy)/np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

In [3]:
import os 
import glob 
import numpy as np
import pandas as pd
import cv2 as cv
from google.colab.patches import cv2_imshow
from skimage import io
from PIL import Image
import matplotlib.pylab as plt
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv('/content/drive/MyDrive/usedataleftarm',header=0)
j=data.iloc[2]
j.to_numpy()
j
data

,head,left leg,right leg,left foot,right foot,chest,center waist,right arm,left hand,right hand
0,-48,-54,-58,-60,-57,-35,-62,-54,-61,-43
1,-48,-54,-58,-60,-57,-36,-62,-54,-61,-48
2,-43,-64,-65,-70,-57,-35,-62,-54,-61,-48
3,-43,-64,-57,-67,-57,-35,-53,-52,-57,-52
4,-43,-64,-57,-67,-57,-40,-53,-52,-57,-45
...,...,...,...,...,...,...,...,...,...,...
1253,-54,-62,-57,-59,-54,-54,-56,-62,-49,-56
1254,-52,-69,-57,-59,-54,-58,-56,-62,-49,-60
1255,-52,-69,-57,-69,-54,-58,-70,-77,-49,-60
1256,-52,-69,-57,-67,-58,-52,-65,-77,-46,-52


In [5]:
data=pd.read_csv('/content/drive/MyDrive/usedataleftarm')
n=5
p=5
arr=[]
for i in range(p):
  row=[]
  a=[]
  b=[]
  for j in range(p):
    a=data.iloc[i]
    a.to_numpy()
    b=data.iloc[j]
    b.to_numpy()
    row.append(distcorr(a,b))
  arr.append(row)
arr

[[1.0,
  0.986421700650728,
  0.9052288859338717,
  0.7780144577198239,
  0.8487950065427526],
 [0.986421700650728,
  1.0,
  0.9148998244230171,
  0.8104996334299587,
  0.8432393969723966],
 [0.9052288859338717,
  0.9148998244230171,
  1.0,
  0.9075884387685237,
  0.932800713220985],
 [0.7780144577198239,
  0.8104996334299587,
  0.9075884387685237,
  1.0,
  0.948963735048155],
 [0.8487950065427526,
  0.8432393969723966,
  0.932800713220985,
  0.948963735048155,
  1.0]]

In [6]:
a=list(data.columns.values)
a

['head',
 'left leg',
 'right leg',
 'left foot',
 'right foot',
 'chest',
 'center waist',
 'right arm',
 'left hand',
 'right hand']

In [7]:
data=pd.read_csv('/content/drive/MyDrive/usedataleftarm')
a=list(data.columns.values)
names1=[]
num=0
for i in range(p):
  for j in range(p):
    if(i!=j):
      if(arr[i][j]>=0.85):
       if a[j] not in names1:
        f=data[a[j]]
        names1.append(a[j])
        num=num+1
names1



['left leg', 'right leg', 'head', 'left foot', 'right foot']

In [8]:
dataafter=pd.DataFrame()
for i in range(num):
  dataafter=dataafter.append(data[names1[i]])
dataafter.set_index(1, inplace = True)
dataafter=dataafter.set_index(0).T.reset_index(drop=True) 
dataafter.columns=names1
dataafter

,left leg,right leg,head,left foot,right foot
0,-64.0,-65.0,-43.0,-70.0,-57.0
1,-64.0,-57.0,-43.0,-67.0,-57.0
2,-64.0,-57.0,-43.0,-67.0,-57.0
3,-52.0,-60.0,-43.0,-59.0,-54.0
4,-52.0,-60.0,-43.0,-64.0,-54.0
...,...,...,...,...,...
1251,-62.0,-57.0,-54.0,-59.0,-54.0
1252,-69.0,-57.0,-52.0,-59.0,-54.0
1253,-69.0,-57.0,-52.0,-69.0,-54.0
1254,-69.0,-57.0,-52.0,-67.0,-58.0


In [9]:
cursize=8
sucsize=cursize
prevsize=10
alpha=-0.005
df1=dataafter.head(cursize)
df2=dataafter.head(prevsize)
currentmean=df1[1:num].mean()
previousmean=df2[1:num].mean()
prod=currentmean*previousmean
var=((currentmean*currentmean)-(previousmean*previousmean))
variation=var/(currentmean*previousmean)

t=0.01+alpha
if(variation[2] > t):
 variationmax=variation.max()
 size=variationmax/variation[2]
 if(cursize>prevsize):
   sucsize=sucsize+(size)
 else:
   sucsize=sucsize-(size) 

sucsize=sucsize*30
sucsize

240

In [10]:
weightdata=dataafter.rolling(int(sucsize)).mean()
#weightdata=data1.rolling(int(300)).mean()
weightdata.dropna()

,left leg,right leg,head,left foot,right foot
239,-59.562500,-58.712500,-49.100000,-64.375000,-64.225000
240,-59.550000,-58.670833,-49.141667,-64.379167,-64.241667
241,-59.537500,-58.679167,-49.183333,-64.358333,-64.258333
242,-59.525000,-58.687500,-49.225000,-64.337500,-64.275000
243,-59.591667,-58.670833,-49.270833,-64.387500,-64.400000
...,...,...,...,...,...
1251,-59.375000,-59.616667,-52.212500,-66.591667,-62.016667
1252,-59.433333,-59.620833,-52.216667,-66.575000,-62.008333
1253,-59.491667,-59.625000,-52.220833,-66.612500,-62.000000
1254,-59.550000,-59.625000,-52.225000,-66.641667,-61.995833


In [11]:
dataafter

,left leg,right leg,head,left foot,right foot
0,-64.0,-65.0,-43.0,-70.0,-57.0
1,-64.0,-57.0,-43.0,-67.0,-57.0
2,-64.0,-57.0,-43.0,-67.0,-57.0
3,-52.0,-60.0,-43.0,-59.0,-54.0
4,-52.0,-60.0,-43.0,-64.0,-54.0
...,...,...,...,...,...
1251,-62.0,-57.0,-54.0,-59.0,-54.0
1252,-69.0,-57.0,-52.0,-59.0,-54.0
1253,-69.0,-57.0,-52.0,-69.0,-54.0
1254,-69.0,-57.0,-52.0,-67.0,-58.0


In [12]:
weightdata=weightdata.reset_index()
weightdata.dropna()
weightdata['index'].iloc[0]
weightdata

,index,left leg,right leg,head,left foot,right foot
0,0,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1251,1251,-59.375000,-59.616667,-52.212500,-66.591667,-62.016667
1252,1252,-59.433333,-59.620833,-52.216667,-66.575000,-62.008333
1253,1253,-59.491667,-59.625000,-52.220833,-66.612500,-62.000000
1254,1254,-59.550000,-59.625000,-52.225000,-66.641667,-61.995833


In [13]:
names=list(dataafter)
names[3]

'left foot'

In [14]:
weightarray=weightdata.to_numpy()
weightarray[3][1]
weightdata.std()
afterarray=dataafter.to_numpy()

In [15]:
dataafter.shape[0]


1256

In [16]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [17]:
t=translate(2,1,10,0,2)
t

0.2222222222222222

In [18]:
col1=['Result']
df=pd.DataFrame(columns=col1)
for j in range(dataafter.shape[0]):
 PF=0
 NF=0
 Pos=0
 for i in range(dataafter.shape[1]-1):
   n=weightdata.shape[0]-int(sucsize)
   TH=weightdata[names[i+1]].std()
   #print(TH)
   AV=afterarray[j][i+1]
   PV=weightarray[(j+int(sucsize))%(dataafter.shape[0]-1)][i+2]
   diff=(abs(AV-PV))
   diff=translate(diff,1,5,0,2)
   diff=np.array(diff).tolist()
   #print(diff)
   if(diff<=TH):
    Pos=Pos+1
    PF=1
    #df=df.append({'NF': 0},ignore_index=True)
   else  :
     NF=1
    #df=df.append({'NF': 1},ignore_index=True)
 avg=0
 for k in range(dataafter.shape[1]-1):
   avg=avg+k
 avg=avg/(dataafter.shape[1]-1)
 #print(avg)
 #print(Pos)
 if(Pos>=avg):
   print('True')
   df=df.append({'Result': 'True'},ignore_index=True)
 else:
   print('False')
   df=df.append({'Result': 'False'},ignore_index=True)


False
False
False
False
True
True
False
True
True
False
False
False
False
True
True
False
True
True
False
False
False
False
False
False
True
False
False
False
True
True
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
True
True
True
False
True
True
False
False
False
False
True
True
False
False
False
True
False
False
True
True
True
True
True
True
False
False
False
True
True
True
True
True
True
False
True
True
True
True
True
False
True
True
True
False
False
True
True
True
False
False
False
True
True
True
False
False
False
False
True
True
True
False
False
False
False
False
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True


In [ ]:
result = pd.concat([dataafter, df], axis=1, join='inner')
result.to_csv('/content/drive/MyDrive/fileleftarm.csv')
result

,left leg,right leg,head,left foot,right foot,Result
0,-64.0,-65.0,-43.0,-70.0,-57.0,False
1,-64.0,-57.0,-43.0,-67.0,-57.0,False
2,-64.0,-57.0,-43.0,-67.0,-57.0,False
3,-52.0,-60.0,-43.0,-59.0,-54.0,False
4,-52.0,-60.0,-43.0,-64.0,-54.0,True
...,...,...,...,...,...,...
1251,-62.0,-57.0,-54.0,-59.0,-54.0,False
1252,-69.0,-57.0,-52.0,-59.0,-54.0,False
1253,-69.0,-57.0,-52.0,-69.0,-54.0,False
1254,-69.0,-57.0,-52.0,-67.0,-58.0,True
